In [10]:
filepath='/content/sample_data/hi_hdtb-ud-test.conllu'

In [17]:
def clean_conllu_file():
    with open(filepath, 'r') as f:
        data = f.readlines()

    output = ''

    for line in data:
        if line == '\n':
            continue

        if line.startswith('# sent_id'):
            output += '\n'
        output += line
    print(output[:100])
    with open('/content/sample_data/DATA/newfile.conllu', 'w+') as f:
        f.write(output)


In [18]:
clean_conllu_file()


# sent_id = test-s1
# text = इसके अतिरिक्त गुग्गुल कुंड, भीम गुफा तथा भीमशिला भी दर्शनीय स्थल हैं ।


In [21]:
#converts a conllu file to json
import pprint
import os
import json
import copy

def conllu_to_json(filepath=None):
    # initialize
    text = []
    # Read conllu file
    with open(filepath, 'r', encoding='utf-8') as f:
        source = f.read()
    sentences = source.strip().split('\n\n')

    for sentence in sentences:
        temp_lines = sentence.strip().split('\n')

        sent = {}
        lines = []
        for line in temp_lines:
            words = line.split(' ')
            # Drop all lines beginning with #
            if words[0] == '#':
                if words[1] == 'sent_id':
                    sent['sent_id'] = words[3]
            else:
                lines.append(line)

        reject_sentence = False
        words = []
        for line in lines:
            words_list = line.split('\t')

            try:
                int(words_list[0])
            except ValueError:
                reject_sentence = True
                break

            word = {
                "id": words_list[0],
                "form": words_list[1],
                "lemma": words_list[2],
                "upostag": words_list[3],
                "xpostag": words_list[4],
                "feats": words_list[5],
                "head": words_list[6],
                "deprel": words_list[7],
                "deps": words_list[8],
                "misc": words_list[9]
            }
            words.append(word)

            if word['deprel'] == 'root':
                words.append({
                    "id": "0",
                    "form": "<ROOT> ",
                    "lemma": "<ROOT>",
                    "upostag": "ROOT",
                    "xpostag": "ROOT",
                    "feats": "_",
                    "head": "-1",
                    "deprel": "_",
                    "deps": "_",
                    "misc": "_"
                })

        if reject_sentence:
            continue

        sent['words'] = words
        text.append(sent)

    with open(filepath.replace('conllu', 'json'), 'w+') as f:
        f.write(json.dumps(text, indent=4))

In [22]:
conllu_to_json('/content/sample_data/DATA/newfile.conllu')